In [1]:
#################################################
# IMDB ETL - Init
#################################################

import pandas as pd
import numpy  as np
###
# Null date conversion fun
def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

In [2]:
import pandas as pd
import numpy  as np

#################################################
# Principals
#################################################

P = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
P["cat_cln"] = np.where (P['category'].str.contains('actor') | P['category'].str.contains('actress')
                         ,'actor'
                         ,P['category'])
P["cat_cln"] = np.where (P['category'].str.contains('self') | P['category'].str.contains('archive_footage')
                         ,'self',P['cat_cln'])


P.head(5)

,tconst,ordering,nconst,category,job,characters,cat_cln
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",self
1,tt0000001,2,nm0005690,director,\N,\N,director
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N,cinematographer
3,tt0000002,1,nm0721526,director,\N,\N,director
4,tt0000002,2,nm1335271,composer,\N,\N,composer


In [3]:
import pandas as pd
import numpy  as np
#################################################
# Titles
#################################################

def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

T = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t'
                 , encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                #, error_bad_lines=False
                #, dtype={'startYear': int }
                 , converters={'startYear':conv,'endYear':conv}                
                ) 
T["flg_doc"]           = np.where (T['genres'].str.contains('Documentary'),'Doc','Fiction')
T["primaryTitle_year"] = T['primaryTitle']  + ' ('+  T['startYear'].map(str)  + ')'
## filter movie 
T = T.loc[T.titleType == 'movie', :]
T.head(10)

C:\Users\cw13\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year
498,tt0000502,movie,Bohemios,Bohemios,0,1905,0,100,\N,Fiction,Bohemios (1905)
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,0,70,"Action,Adventure,Biography",Fiction,The Story of the Kelly Gang (1906)
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,0,90,Drama,Fiction,The Prodigal Son (1907)
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,0,\N,Drama,Fiction,Robbery Under Arms (1907)
625,tt0000630,movie,Hamlet,Amleto,0,1908,0,\N,Drama,Fiction,Hamlet (1908)
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,0,\N,Drama,Fiction,Don Quijote (1908)
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,0,120,"Adventure,Fantasy",Fiction,The Fairylogue and Radio-Plays (1908)
783,tt0000793,movie,Andreas Hofer,Andreas Hofer,0,1909,0,\N,Drama,Fiction,Andreas Hofer (1909)
804,tt0000814,movie,La bocana de Mar Chica,La bocana de Mar Chica,0,1909,0,\N,\N,Fiction,La bocana de Mar Chica (1909)
828,tt0000838,movie,A Cultura do Cacau,A Cultura do Cacau,0,1909,0,\N,\N,Fiction,A Cultura do Cacau (1909)


In [4]:
var_list = ('tconst','primaryTitle' , 'titleType')
#D = newdf.loc[:,('const','title', 'dat_per', 'title_type')]
D = T.loc[:, var_list]
D.head()
pd.get_dummies(D, columns=['titleType']).head(3)


,tconst,primaryTitle,titleType_movie
498,tt0000502,Bohemios,1
570,tt0000574,The Story of the Kelly Gang,1
587,tt0000591,The Prodigal Son,1


In [5]:
#################################################
# Names
#################################################
    
N = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                 , converters={'birthYear':conv,'deathYear':conv}         
                )


#N["primaryName_year"] = N['primaryName']  + ' ('+  N['birthYear'].map(str) + ' - ' +  N['deathYear'].map(str)  + ')'

N["primaryName_year"] = N['primaryName']  + np.where(N['birthYear']==0, ' ',  ' ('+  N['birthYear'].map(str) + ' - '  +  np.where (N['deathYear'] == 0, ' ',  N['deathYear'].map(str)) + ')')
N.head(3)


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,primaryName_year
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0053137,tt0072308,tt0050419",Fred Astaire (1899 - 1987)
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0117057,tt0037382,tt0038355",Lauren Bacall (1924 - 2014)
2,nm0000003,Brigitte Bardot,1934,0,"actress,soundtrack,music_department","tt0049189,tt0056404,tt0057345,tt0054452",Brigitte Bardot (1934 - )


In [6]:
#################################################
# Ratings
#################################################
R = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
R.rename(columns = {  "averageRating": "tconst_imdb_rating"
                      , "numVotes":   "tconst_imdb_num_votes"
                      , "Const": "tconst"}
                      , inplace = True)
R.tail(3)

,tconst,tconst_imdb_rating,tconst_imdb_num_votes
1250781,tt9916730,8.4,6
1250782,tt9916766,6.7,20
1250783,tt9916778,7.2,35


In [7]:
############################################ 
### import my Ratings 
############################################ 
myR = pd.DataFrame()
myR = pd.read_csv ("data/ratings.csv", encoding='latin-1')
myR.rename(columns = {  "Your Rating": "tconst_your_rating"
                      , "Num Votes":   "tconst_your_num_votes"
                      , "Const": "tconst"}
                      , inplace = True)
myR.tail(3)

,tconst,tconst_your_rating,Date Rated,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,tconst_your_num_votes,Release Date,Directors
864,tt0989757,6,2015-10-11,Dear John,https://www.imdb.com/title/tt0989757/,movie,6.3,108.0,2010,"Drama, Mystery, Romance, Thriller, War",147548.0,2010-01-24,Lasse HallstrÃ¶m
865,tt0099699,7,2013-01-01,Green Card,https://www.imdb.com/title/tt0099699/,movie,6.3,107.0,1990,"Comedy, Drama, Romance",24352.0,1990-12-23,Peter Weir
866,tt0099703,6,2011-09-03,The Grifters,https://www.imdb.com/title/tt0099703/,movie,6.9,110.0,1990,"Crime, Drama, Thriller",27901.0,1990-09-14,Stephen Frears


In [8]:
#################################################
# Join
#################################################
from datetime import datetime
             
imdb = pd.merge(    P
                  , T 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(     N.loc[:, ('nconst','primaryProfession', 'primaryName', 'birthYear', 'deathYear', 'primaryName_year')]
                  , imdb
                  , how = 'inner', left_on="nconst", right_on="nconst"
                 )
imdb = pd.merge(     R.loc[:, ('tconst','tconst_imdb_rating', 'tconst_imdb_num_votes')]
                  , imdb
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(    myR.loc[:, ('tconst','tconst_your_rating')]
                  , imdb
                  , how = 'right', left_on="tconst", right_on="tconst"
                 )
imdb['rate_diff']           = imdb['tconst_your_rating'] - imdb['tconst_imdb_rating']
##imdb["age_crew"]            = int(imdb["birthYear_cln"]) -  int(imdb["startYear_cln"])
imdb["last_refresh"]        = datetime.now().strftime('%Y%m%d%H%M%S')
print(P.shape, imdb.shape)


(50630284, 7) (2487705, 27)


In [9]:
imdb[imdb['tconst'] == "tt1745960"].head(15)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,rate_diff,last_refresh
1747344,tt1745960,NaN,8.7,98820,nm0000124,actress,Jennifer Connelly,1970,0,Jennifer Connelly (1970 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747345,tt1745960,NaN,8.7,98820,nm0000129,"actor,producer,soundtrack",Tom Cruise,1962,0,Tom Cruise (1962 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747346,tt1745960,NaN,8.7,98820,nm0000174,"actor,soundtrack,producer",Val Kilmer,1959,0,Val Kilmer (1959 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747347,tt1745960,NaN,8.7,98820,nm0143596,writer,Jim Cash,1941,2000,Jim Cash (1941 - 2000),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747348,tt1745960,NaN,8.7,98820,nm0185976,"writer,actor,producer",Peter Craig,1969,0,Peter Craig (1969 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747349,tt1745960,NaN,8.7,98820,nm0258390,"writer,camera_department,producer",Jack Epps Jr.,1949,0,Jack Epps Jr. (1949 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747350,tt1745960,NaN,8.7,98820,nm0472567,"writer,producer,executive",Ehren Kruger,1972,0,Ehren Kruger (1972 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747351,tt1745960,NaN,8.7,98820,nm1098479,"writer,producer,miscellaneous",Justin Marks,0,0,Justin Marks,...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747352,tt1745960,NaN,8.7,98820,nm1886602,"actor,producer,soundtrack",Miles Teller,1987,0,Miles Teller (1987 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211
1747353,tt1745960,NaN,8.7,98820,nm2676052,"director,writer,producer",Joseph Kosinski,1974,0,Joseph Kosinski (1974 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211


In [10]:
#################################################
# add row number forfilmmakers with more than one release per year
#################################################
imdb['RN'] = imdb.sort_values(['nconst', 'tconst'], ascending=[True, False]) \
             .groupby(['nconst', 'startYear']) \
             .cumcount() + 1
imdb.tail(10)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,rate_diff,last_refresh,RN
2487695,tt9916538,NaN,8.3,6,nm8678236,actress,Lala Karmela,1985,0,Lala Karmela (1985 - ),...,0,2019,0,123,Drama,Fiction,Kuambil Lagi Hatiku (2019),NaN,20220606120211,1
2487696,tt9916730,NaN,8.4,6,nm0059461,"actor,music_department",Sunil Barve,0,0,Sunil Barve,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487697,tt9916730,NaN,8.4,6,nm10538612,NaN,Kiran Gawade,0,0,Kiran Gawade,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487698,tt9916730,NaN,8.4,6,nm10538613,producer,Abhishek Jathar,0,0,Abhishek Jathar,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487699,tt9916730,NaN,8.4,6,nm10538614,producer,Ujjwala Gawde,0,0,Ujjwala Gawde,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487700,tt9916730,NaN,8.4,6,nm13233318,"actor,production_manager",Ganesh Vasant Patil,0,0,Ganesh Vasant Patil,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487701,tt9916730,NaN,8.4,6,nm1957275,"cinematographer,camera_department,producer",Suresh Deshmane,0,0,Suresh Deshmane,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487702,tt9916730,NaN,8.4,6,nm4852679,actor,Bhushan Pradhan,0,0,Bhushan Pradhan,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487703,tt9916730,NaN,8.4,6,nm6096005,actor,Devadhar Archit,0,0,Devadhar Archit,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1
2487704,tt9916730,NaN,8.4,6,nm9785908,"assistant_director,editor",Rohita More,0,0,Rohita More,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220606120211,1


In [11]:
#################################################
#remove rows not needed 
#################################################

X = imdb[imdb['category']!= "production_designer"] 
X = X[X['category']!= "archive_sound"] 
#X = X[X['category']!= "archive_footage"]
print(X.shape, imdb.shape)

#X.sort_values(by=['tconst', 'averageRating'], ascending=False)
#################################################
# export to file
#################################################
X.to_csv('data/imdb.csv'  ,encoding='utf-8')

(2461434, 28) (2487705, 28)


In [12]:
X[X['tconst'] == "tt1745960"].head(15)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,rate_diff,last_refresh,RN
1747344,tt1745960,NaN,8.7,98820,nm0000124,actress,Jennifer Connelly,1970,0,Jennifer Connelly (1970 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747345,tt1745960,NaN,8.7,98820,nm0000129,"actor,producer,soundtrack",Tom Cruise,1962,0,Tom Cruise (1962 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747346,tt1745960,NaN,8.7,98820,nm0000174,"actor,soundtrack,producer",Val Kilmer,1959,0,Val Kilmer (1959 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747347,tt1745960,NaN,8.7,98820,nm0143596,writer,Jim Cash,1941,2000,Jim Cash (1941 - 2000),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747348,tt1745960,NaN,8.7,98820,nm0185976,"writer,actor,producer",Peter Craig,1969,0,Peter Craig (1969 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,2
1747349,tt1745960,NaN,8.7,98820,nm0258390,"writer,camera_department,producer",Jack Epps Jr.,1949,0,Jack Epps Jr. (1949 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747350,tt1745960,NaN,8.7,98820,nm0472567,"writer,producer,executive",Ehren Kruger,1972,0,Ehren Kruger (1972 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747351,tt1745960,NaN,8.7,98820,nm1098479,"writer,producer,miscellaneous",Justin Marks,0,0,Justin Marks,...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747352,tt1745960,NaN,8.7,98820,nm1886602,"actor,producer,soundtrack",Miles Teller,1987,0,Miles Teller (1987 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1
1747353,tt1745960,NaN,8.7,98820,nm2676052,"director,writer,producer",Joseph Kosinski,1974,0,Joseph Kosinski (1974 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),NaN,20220606120211,1


In [13]:
#################################################
# Distinct NCONST aggregation
#################################################
import numpy as np
N_avg = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.mean]                   
                   , values = ["tconst_imdb_rating", "tconst_your_rating"]
                   , fill_value = 0
             )
N_sum = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.sum]                   
                   , values = [ "tconst_imdb_num_votes"]
                   , fill_value = 0
             )
N_max = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.max]                   
                   , values = [ "startYear"]
                   , fill_value = 0
             )
N_cnt = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ 'count']  
                   , values = ["tconst"]
                   , fill_value = 0
             )

N_unique = pd.merge(   N_sum
                     , N_avg 
                     , how = 'inner', left_on=  [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_cnt
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_max
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )

In [14]:
# lst = ['nm0000001', 'nm0000002']
# N_unique =N_unique.query('nconst in @lst')
N_unique.to_csv('data/nconst.csv' , header=0 , encoding='utf-8')

In [30]:
############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls560308483/    Cannes 2022: Official Selections
## https://www.imdb.com/list/ls560342137     Cannes 75

fst = pd.DataFrame()
fst = pd.read_csv ("data/Cannes 2022 Official Selections.csv", encoding='latin-1')
fst.rename(columns = {  "Your Rating": "your_rating"
                      , "Const": "tconst"}
                      , inplace = True)
fst['festival_name'] = 'Cannes'
fst['festival_year'] = '2022'


############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls565715845   Yesh 2022
fst1 = pd.DataFrame()
fst1 = pd.read_csv ("data/Yesh2022.csv", encoding='latin-1')
fst1.rename(columns = {  "Your Rating": "your_rating"
                      , "Const": "tconst"} 
                      , inplace = True)
fst1['festival_name'] = 'Yesh!'
fst1['festival_year'] = '2022'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls565715845   Yesh 2022
fst2 = pd.DataFrame()
fst2 = pd.read_csv ("data/Zurich Film Festival 2021.csv", encoding='latin-1')
fst2.rename(columns = {  "Your Rating": "your_rating"
                      , "Const": "tconst"}
                      , inplace = True)
fst2['festival_name'] = 'ZFF'
fst2['festival_year'] = '2021'

union = pd.concat([fst, fst1])
union = pd.concat([union, fst2]) 
union.tail(3)

C:\Users\cw13\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\cw13\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Created,Date Rated,Description,Directors,Genres,IMDb Rating,Modified,Num Votes,Position,Release Date,Runtime (mins),Title,Title Type,URL,Year,festival_name,festival_year,tconst,your_rating
136,2021-10-10,NaN,NaN,Marc Forster,"Action, Adventure, Thriller",6.6,2021-10-10,428043.0,137,2008-10-29,106.0,Quantum of Solace,movie,https://www.imdb.com/title/tt0830515/,2008,ZFF,2021,tt0830515,NaN
137,2021-10-10,NaN,NaN,Martin Campbell,"Action, Adventure, Thriller",8.0,2021-10-10,620065.0,138,2006-11-14,144.0,Casino Royale,movie,https://www.imdb.com/title/tt0381061/,2006,ZFF,2021,tt0381061,NaN
138,2021-10-27,NaN,NaN,Philippe Weibel,Drama,9.5,2021-10-27,30.0,139,2020-03-01,105.0,The Art of Love,movie,https://www.imdb.com/title/tt11418284/,2020,ZFF,2021,tt11418284,NaN


In [53]:
          
Xfst = pd.merge(    X
                  , union 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
Xfst.to_csv('data/Xfst.csv' , header=1 , encoding='utf-8')
Xfst.head(3)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,Position,Release Date,Runtime (mins),Title,Title Type,URL,Year,festival_name,festival_year,your_rating
0,tt10360862,5.0,7.0,5515,nm0100460,actor,Koen De Bouw,1964,0,Koen De Bouw (1964 - ),...,22,2020-09-04,104.0,The Man Who Sold His Skin,movie,https://www.imdb.com/title/tt10360862/,2020,ZFF,2021,NaN
1,tt10360862,5.0,7.0,5515,nm0252677,"actress,assistant_director",Darina Al Joundi,0,0,Darina Al Joundi,...,22,2020-09-04,104.0,The Man Who Sold His Skin,movie,https://www.imdb.com/title/tt10360862/,2020,ZFF,2021,NaN
2,tt10360862,5.0,7.0,5515,nm0439003,producer,Thanassis Karathanos,0,0,Thanassis Karathanos,...,22,2020-09-04,104.0,The Man Who Sold His Skin,movie,https://www.imdb.com/title/tt10360862/,2020,ZFF,2021,NaN


In [67]:

# duplicate for rating dimension


# Xfst2 = pd.DataFrame({'Xfst':['tconst','nconst'],})
# Xfst2 = pd.DataFrame({'Xfst':['tconst','nconst'],})
Xfst2 = Xfst.copy()   # panda refernces tables only, so explicit copy required
  
Xfst2['rating_category']           = 'IMDB'
Xfst2['tconst_num_votes']          = Xfst2['tconst_imdb_num_votes']   
Xfst2['tconst_rating']             = Xfst2['tconst_imdb_rating']   
Xfst2.head(3)

Xfst3 = Xfst.copy()
  
Xfst3['rating_category']           = 'Mine'
Xfst3['tconst_num_votes']          = 1   
Xfst3['tconst_rating']             = np.where(Xfst3['tconst_your_rating'].isnull(), 0, Xfst3['tconst_your_rating'] )
Xfst3.head(3)
 


rating_duplicate = pd.concat([Xfst2, Xfst3], ignore_index=True) 
rating_duplicate.to_csv('data/rating_duplicate.csv' , header=1 , encoding='utf-8')

In [68]:
rating_duplicate.tail(4)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,Title,Title Type,URL,Year,festival_name,festival_year,your_rating,rating_category,tconst_num_votes,tconst_rating
3988,tt9812474,NaN,6.3,22912,nm5762091,"camera_department,special_effects,director",Valdimar Jóhannsson,1978,0,Valdimar Jóhannsson (1978 - ),...,Lamb,movie,https://www.imdb.com/title/tt9812474/,2021,ZFF,2021,NaN,Mine,1,0.0
3989,tt9812474,NaN,6.3,22912,nm6227084,"miscellaneous,producer,script_department",Jan Naszewski,0,0,Jan Naszewski,...,Lamb,movie,https://www.imdb.com/title/tt9812474/,2021,ZFF,2021,NaN,Mine,1,0.0
3990,tt9883832,NaN,7.4,37,nm0000309,"soundtrack,composer,actor",David Bowie,1947,2016,David Bowie (1947 - 2016),...,Moonage Daydream,movie,https://www.imdb.com/title/tt9883832/,2022,Cannes,2022,NaN,Mine,1,0.0
3991,tt9883832,NaN,7.4,37,nm0605137,"producer,director,writer",Brett Morgen,1968,0,Brett Morgen (1968 - ),...,Moonage Daydream,movie,https://www.imdb.com/title/tt9883832/,2022,Cannes,2022,NaN,Mine,1,0.0


In [17]:
print(fst.shape, Xfst.shape)
N_unique.tail(3)

(66, 17) (2016, 46)


sum               mean  \
                                    tconst_imdb_num_votes tconst_imdb_rating   
nconst    primaryName_year cat_cln                                             
nm9993650 Marcin Balcerak  actor                      126                5.4   
nm9993693 Apsara Rani      actor                      407                3.8   
nm9993694 Chinmay Mishra   director                    30                8.3   

                                                        count      amax  
                                    tconst_your_rating tconst startYear  
nconst    primaryName_year cat_cln                                       
nm9993650 Marcin Balcerak  actor                   0.0      1      2018  
nm9993693 Apsara Rani      actor                   0.0      2      2021  
nm9993694 Chinmay Mishra   director                0.0      2      2022

In [18]:
Xfst.tail(3)
Xfst[Xfst['tconst'] == "tt1745960"].head(15)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,Position,Release Date,Runtime (mins),Title,Title Type,URL,Year,festival_name,festival_year,your_rating
1419,tt1745960,NaN,8.7,98820.0,nm0000124,actress,Jennifer Connelly,1970.0,0.0,Jennifer Connelly (1970 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1420,tt1745960,NaN,8.7,98820.0,nm0000129,"actor,producer,soundtrack",Tom Cruise,1962.0,0.0,Tom Cruise (1962 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1421,tt1745960,NaN,8.7,98820.0,nm0000174,"actor,soundtrack,producer",Val Kilmer,1959.0,0.0,Val Kilmer (1959 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1422,tt1745960,NaN,8.7,98820.0,nm0143596,writer,Jim Cash,1941.0,2000.0,Jim Cash (1941 - 2000),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1423,tt1745960,NaN,8.7,98820.0,nm0185976,"writer,actor,producer",Peter Craig,1969.0,0.0,Peter Craig (1969 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1424,tt1745960,NaN,8.7,98820.0,nm0258390,"writer,camera_department,producer",Jack Epps Jr.,1949.0,0.0,Jack Epps Jr. (1949 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1425,tt1745960,NaN,8.7,98820.0,nm0472567,"writer,producer,executive",Ehren Kruger,1972.0,0.0,Ehren Kruger (1972 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1426,tt1745960,NaN,8.7,98820.0,nm1098479,"writer,producer,miscellaneous",Justin Marks,0.0,0.0,Justin Marks,...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1427,tt1745960,NaN,8.7,98820.0,nm1886602,"actor,producer,soundtrack",Miles Teller,1987.0,0.0,Miles Teller (1987 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
1428,tt1745960,NaN,8.7,98820.0,nm2676052,"director,writer,producer",Joseph Kosinski,1974.0,0.0,Joseph Kosinski (1974 - ),...,42,2022-05-18,131.0,Top Gun: Maverick,movie,https://www.imdb.com/title/tt1745960/,2022,Cannes,2022,NaN
